In [1]:
# We used code from this tutorial:
# https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Collaborative%20Filtering%20Model%20with%20TensorFlow.ipynb
# And we also used code from this tutorial:
# https://medium.com/@connectwithghosh/recommender-system-on-the-movielens-using-an-autoencoder-using-tensorflow-in-python-f13d3e8d600d
# Then, we integrated these two tutorials and edited the code from each of them in order to create a recommender that allows us to recommend a top 10 list of movies without needing to retrain for each new user

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [3]:
rating = pd.read_csv('/Users/blakemyers/Desktop/data/ratings.csv', error_bad_lines=False, encoding='latin-1')

In [4]:
movie = pd.read_csv("/Users/blakemyers/Desktop/data/movies.csv", error_bad_lines=False, encoding='latin-1')

In [5]:
movie_rating = pd.merge(rating, movie, on = 'movieId')

In [6]:
cols = ['timestamp']

In [7]:
movie_rating.drop(cols, axis=1, inplace=True)

In [8]:
numrate_movie = movie_rating.groupby("title")["rating"].count().reset_index()

In [9]:
numrate_movie.rename({"rating": "ratecount_movie"}, axis=1, inplace=True)

In [10]:
numrate_movie = numrate_movie.query("ratecount_movie >= 20")

In [11]:
ratings20plus = pd.merge(numrate_movie, movie_rating, on = 'title', how = 'inner')

In [12]:
numrate_user = ratings20plus.groupby("userId")["rating"].count().reset_index()

In [13]:
numrate_user.rename({"rating": "ratecount_user"}, axis=1, inplace=True)

In [14]:
numrate_user = numrate_user.query("ratecount_user >= 20")

In [15]:
ur20plus= pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")

In [16]:
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled

In [17]:
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)

In [18]:
X_train, X_test = train_test_split(user_movie_matrix, train_size=0.8)

In [19]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [20]:
num_input = ur20plus['title'].nunique()
n_nodes_inpl = num_input  
n_nodes_hl1  = 256  
n_nodes_outl = num_input  
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}
output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl]))}

In [21]:
input_layer = tf.placeholder('float', [None, num_input])
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,hidden_1_layer_vals['weights']))
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
output_true = tf.placeholder('float', [None, num_input])
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
learn_rate = 0.1
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [22]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
batch_size = 100
hm_epochs =200
tot_images = X_train.shape[0]

In [23]:
for epoch in range(hm_epochs):
    epoch_loss = 0
    
    for i in range(int(tot_images/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer,\
               feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer,\
                   feed_dict={input_layer:X_test})
        
    if epoch <= 10 or epoch >= 190:
        print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))
        print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)
    elif epoch == 11:
        print('.......................')
    else:
        continue

MSE train 85.26557522124689 MSE test 85.32654965854096
Epoch 0 / 200 loss: 392.18053436279297
MSE train 71.84931197075936 MSE test 71.96227213184854
Epoch 1 / 200 loss: 319.0783920288086
MSE train 62.867976500428036 MSE test 63.02341941149882
Epoch 2 / 200 loss: 272.68739318847656
MSE train 56.528467517934594 MSE test 56.72587973875652
Epoch 3 / 200 loss: 241.0486946105957
MSE train 51.82830375493313 MSE test 52.09616366664766
Epoch 4 / 200 loss: 218.34769821166992
MSE train 48.205318933451856 MSE test 48.574907126192606
Epoch 5 / 200 loss: 201.27299880981445
MSE train 45.34290346301186 MSE test 45.818046594924716
Epoch 6 / 200 loss: 187.97752380371094
MSE train 43.02655303050838 MSE test 43.60500727810276
Epoch 7 / 200 loss: 177.3773651123047
MSE train 41.120503684459464 MSE test 41.79050338061943
Epoch 8 / 200 loss: 168.72731399536133
MSE train 39.53200104395983 MSE test 40.27699722727292
Epoch 9 / 200 loss: 161.570556640625
MSE train 38.187229356908595 MSE test 38.993875042200095
Ep

In [24]:
#retrieve the top 10 recommendations for a new user who rated one movie, i.e., "Aliens (1986)", with a 5/5.
userCol = ur20plus["userId"]
max_value = userCol.max()
new_userId = max_value + 1
ur20plus= pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")
ur20plus = ur20plus.append(pd.DataFrame([["Aliens (1986)",1,new_userId,1,5,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)
list(top_ten_ranked.loc[top_ten_ranked['userId'] == new_userId]['title'].values)

['Drive (2011)',
 'Hudsucker Proxy, The (1994)',
 'Omen, The (1976)',
 'Tombstone (1993)',
 'Cool Runnings (1993)',
 'Splash (1984)',
 'Grifters, The (1990)',
 'Traffic (2000)',
 'Fighter, The (2010)',
 'Fish Called Wanda, A (1988)']

In [25]:
#retrieve the top 10 recommendations for a new user who rated one movie, i.e., "(500) Days of Summer (2009)", with a 3/5.
userCol = ur20plus["userId"]
max_value = userCol.max()
new_userId = max_value + 1
ur20plus= pd.merge(ratings20plus, numrate_user, on = "userId", how = "inner")
ur20plus = ur20plus.append(pd.DataFrame([["(500) Days of Summer (2009)",1,new_userId,1,3,"genre",1]], columns =ur20plus.columns), ignore_index=True)
scaler = MinMaxScaler()
ur20plus['rating'] = ur20plus['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ur20plus['rating'].values.reshape(-1,1)))
ur20plus['rating'] = rating_scaled
ur20plus = ur20plus.drop_duplicates(['userId', 'title'])
user_movie_matrix = ur20plus.pivot(index='userId', columns='title', values='rating')
user_movie_matrix.fillna(0, inplace=True)
pred_data = pd.DataFrame()
preds = sess.run(output_layer, feed_dict={input_layer: user_movie_matrix})
pred_data = pred_data.append(pd.DataFrame(preds))
pred_data = pred_data.stack().reset_index(name='rating')
pred_data.columns = ['userId', 'title', 'rating']
users = user_movie_matrix.index.tolist()
movies = user_movie_matrix.columns.tolist()
pred_data['userId'] = pred_data['userId'].map(lambda value: users[value])
pred_data['title'] = pred_data['title'].map(lambda value: movies[value])
keys = ['userId', 'title']
index_1 = pred_data.set_index(keys).index
index_2 = ur20plus.set_index(keys).index
top_ten_ranked = pred_data[~index_1.isin(index_2)]
top_ten_ranked = top_ten_ranked.sort_values(['userId', 'rating'], ascending=[True, False])
top_ten_ranked = top_ten_ranked.groupby('userId').head(10)
list(top_ten_ranked.loc[top_ten_ranked['userId'] == new_userId]['title'].values)

['eXistenZ (1999)',
 "Singin' in the Rain (1952)",
 'Mystery Men (1999)',
 'Departed, The (2006)',
 'Like Water for Chocolate (Como agua para chocolate) (1992)',
 'Big Trouble in Little China (1986)',
 'Splash (1984)',
 'Close Encounters of the Third Kind (1977)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Young Frankenstein (1974)']